In [1]:
def download_data():
    """Download and extract the training data."""
    import urllib
    from zipfile import ZipFile
    # download data
    data_file = './fowl_data.zip'
    download_url = 'https://azureopendatastorage.blob.core.windows.net/testpublic/temp/fowl_data.zip'
    urllib.request.urlretrieve(download_url, filename=data_file)

    # extract files
    with ZipFile(data_file, 'r') as zip:
        print('extracting files...')
        zip.extractall()
        print('finished extracting')
        data_dir = zip.namelist()[0]

    # delete zip file
    os.remove(data_file)
    return data_dir

In [2]:
download_data()

extracting files...
finished extracting


'fowl_data/'

In [1]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml import command
from azure.ai.ml import Input
from azure.identity import DefaultAzureCredential

In [2]:
# Enter details of your AML workspace
subscription_id = ""
resource_group = ""
workspace = ""
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

In [3]:
job = command(
    code="./src",  # local path where the code is stored
    command="pip install -r requirements.txt && python train.py --data_dir ${{inputs.data_dir}} --num_epochs ${{inputs.num_epochs}} --learning_rate ${{inputs.learning_rate}} --momentum ${{inputs.momentum}} --output_dir ${{inputs.output_dir}}",
    inputs={
        "data_dir": Input(
            type="uri_folder",
            path="./fowl_data",
        ),
        "momentum": 10,
        "learning_rate": 0.1,
        "output_dir":'./outputs',
        "num_epochs":30,
    },
    environment="AzureML-pytorch-1.7-ubuntu18.04-py37-cuda11-gpu@latest",
    compute="gpu-cluster",
    display_name="pytorch-birds",
    description="Train a neural network with PyTorch.",
)

In [4]:
returned_job = ml_client.create_or_update(job)

In [5]:
ml_client.jobs.stream(returned_job.name)

RunId: bold_hamster_94qv6hvxzn
Web View: https://ml.azure.com/runs/bold_hamster_94qv6hvxzn?wsid=/subscriptions/225d6361-069a-4dfb-9bbb-3ebb42663de0/resourcegroups/Learninggroup/workspaces/amltestcg

Streaming user_logs/std_log.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 86.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 kB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 42.0 MB/s eta 0:00:00
Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth

  0%|          | 0.00/44.7M [00:00<?, ?B/s]
 36%|███▌      | 16.1M/44.7M [00:00<00:00, 169MB/s]
 75%|███████▍  | 33.4M/44.7M [00:00<00:00, 176MB/s]
10

In [3]:
#returned_job.outputs

In [2]:
#returned_job.id.split('/')[-1]

In [1]:
#returned_job.name

In [6]:
from azure.ai.ml.constants import AssetTypes


In [7]:
from azure.ai.ml.entities import Model
#from azure.ai.ml.constants import ModelType
from azure.ai.ml.constants import AssetTypes
jobid=returned_job.name
run_model = Model(
    #path="azureml://subscriptions/XXXXXXXXXXXXXXXXXXXXX/resourceGroups/XXXXXXXXX/workspaces/XXXXXXXXXXXXX/jobs/XXXXXXXXXXXX/outputs/artifacts/paths/model/",
    #Users could also use, "azureml://jobs/XXXXXXXXXXXXXXXXXXXXXXXXX/outputs/artifacts/paths/model/" as a shorthand to the same location
    path="azureml://jobs/"+jobid+"/outputs/artifacts/paths/model/",
    #path="azureml://datastores/workspaceblobstore/paths/model",
    name="pytorchbirdsmodel",
    description="Model created from run.",
    type=AssetTypes.MLFLOW_MODEL
)
# Uncomment after adding required details above
ml_client.models.create_or_update(run_model) 

Model({'job_name': 'elated_plow_4xn8ts3rvt', 'is_anonymous': False, 'auto_increment_version': False, 'name': 'pytorchbirdsmodel', 'description': 'Model created from run.', 'tags': {}, 'properties': {}, 'id': '/subscriptions/225d6361-069a-4dfb-9bbb-3ebb42663de0/resourceGroups/Learninggroup/providers/Microsoft.MachineLearningServices/workspaces/amltestcg/models/pytorchbirdsmodel/versions/4', 'base_path': './', 'creation_context': <azure.ai.ml._restclient.v2022_05_01.models._models_py3.SystemData object at 0x7fe24ae0d9e8>, 'serialize': <msrest.serialization.Serializer object at 0x7fe2569d3748>, 'version': '4', 'latest_version': None, 'path': 'azureml://subscriptions/225d6361-069a-4dfb-9bbb-3ebb42663de0/resourceGroups/Learninggroup/workspaces/amltestcg/datastores/workspaceartifactstore/paths/ExperimentRun/dcid.elated_plow_4xn8ts3rvt/model', 'utc_time_created': None, 'flavors': {'python_function': {'data': 'data', 'env': 'conda.yaml', 'loader_module': 'mlflow.pytorch', 'pickle_module_name':